In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [ ]:
# Define the root directories for your train and test datasets
train_dir = 'C:/Users/denis/Desktop/HNS/projekt1/Dataset_Znacky/Train-adj-train-test/train'
test_dir = 'C:/Users/denis/Desktop/HNS/projekt1/Dataset_Znacky/Train-adj-train-test/test'

# Create a list of unique class labels from the train dataset
unique_labels = sorted(os.listdir(train_dir))

# Create a mapping from class labels to sequential integers
label_to_int = {label: i for i, label in enumerate(unique_labels)}

# Define transforms for data augmentation and normalization
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(299),  # InceptionV3's input size is 299x299
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, label_to_int, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []

        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)

            if os.path.isdir(class_dir):
                images = os.listdir(class_dir)
                for img in images:
                    image_path = os.path.join(class_dir, img)
                    label = label_to_int[class_name]
                    self.samples.append((image_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
# Create custom datasets for train and test
train_dataset = CustomDataset(train_dir, label_to_int, data_transforms)
test_dataset = CustomDataset(test_dir, label_to_int, data_transforms)

# Create data loaders for training and testing
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Load a pretrained InceptionV3 model
model = models.inception_v3(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Now, unfreeze the parameters in the final layer so that they can be trained
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(unique_labels))  # Make sure `unique_labels` is defined and correct


In [ ]:
# Set the loss function and optimizer
import torch.optim as optim
import torch
import matplotlib.pyplot as plt

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)  # 'model.fc.parameters()' as an example for the last layer

# Choose the right device based on availability of CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 20
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0
    with torch.no_grad():  # Gradient computation is not needed for validation
        for val_inputs, val_labels in test_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()

    val_loss = val_running_loss / len(test_loader)
    val_losses.append(val_loss)

    # Print losses
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}")

# Plotting training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()









KeyboardInterrupt: ignored

In [ ]:
# Test the model on the testing data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

KeyboardInterrupt: 